# Image segmentation

## Data preprocessing

In [ ]:
import numpy as np
import os
import time
import cv2
#import nibabel as nib
import pdb
from matplotlib import pyplot as plt
import nibabel as nib
from nibabel.testing import data_path

In [ ]:
def truncated_range(img):
    max_hu = 384
    min_hu = -384
    img[np.where(img > max_hu)] = max_hu
    img[np.where(img < min_hu)] = min_hu
    return (img - min_hu) / (max_hu - min_hu) * 255.


In [ ]:
path = './train/'
save_path = './data/data_npy/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

files = os.listdir(path)
count = 0
print('begin processing data')

means = []
stds = []


for i, volume in enumerate(files):
    total_imgs = []
    cur_file = os.path.join(path, volume)
    print(i, cur_file)
    cur_save_path = os.path.join(save_path, volume)
    if not os.path.exists(cur_save_path):
        os.makedirs(cur_save_path)
    img = nib.load(os.path.join(cur_file, volume + '.nii'))
    img = np.array(img.get_data())
    label = nib.load(os.path.join(cur_file, 'GT.nii'))
    label = np.array(label.get_data())
    img = truncated_range(img)
    for idx in range(img.shape[2]):
        if idx == 0 or idx == img.shape[2] - 1:
            continue
        # 2.5D data, using adjacent 3 images
        cur_img = img[:, :, idx - 1:idx + 2].astype('uint8')
        total_imgs.append(cur_img)
        cur_label = label[:, :, idx].astype('uint8')
        count += 1
        np.save(os.path.join(cur_save_path,volume + '_' + str(idx) + '_image.npy'), cur_img)
        np.save(os.path.join(cur_save_path,volume + '_' + str(idx) + '_label.npy'), cur_label)
    
    total_imgs = np.stack(total_imgs, 3) / 255.
    means.append(np.mean(total_imgs))
    stds.append(np.std(total_imgs))


print('data mean is %f' % np.mean(means))
print('data std is %f' % np.std(stds))
print('total data size is %f' % count)
print('processing data end !')

In [ ]:
for i in range (1,41):
    j=0
    l=1
    if(i<10):
        os.mkdir('./testing/Patient_0'+str(i))
        for k in os.listdir('./data/data_npy/Patient_0'+str(i)):
            j=j+1
    else:
        os.mkdir('./testing/Patient_'+str(i))
        for k in os.listdir('./data/data_npy/Patient_'+str(i)):
            j=j+1
    while(l<=(j/2)):
        if(i<10):
            lbl_array = np.load('./data/data_npy/Patient_0'+str(i)+'/Patient_0'+str(i)+'_'+str(l)+'_label.npy')
            img = Image.fromarray(lbl_array)
            matplotlib.image.imsave('./testing/Patient_0'+str(i)+'/Patient_0'+str(i)+'_'+str(l)+'_label.png', img)
        else:
            lbl_array = np.load('./data/data_npy/Patient_'+str(i)+'/Patient_'+str(i)+'_'+str(l)+'_label.npy')
            img = Image.fromarray(lbl_array)
            matplotlib.image.imsave('./testing/Patient_'+str(i)+'/Patient_'+str(i)+'_'+str(l)+'_label.png', img)
        l=l+1

    


In [ ]:
for i in range(41,61):
    j=0
    l=1
    os.mkdir('./testing_2d/Patient_'+str(i))
    for k in os.listdir('./data_test/data_npy/Patient_'+str(i)+'.nii'):
        j=j+1
    while(l<=(j)):
        img_array = np.load('./data_test/data_npy/Patient_'+str(i)+'.nii/Patient_'+str(i)+'.nii_'+str(l)+'_image.npy')
        matplotlib.image.imsave('./testing_2d/Patient_'+str(i)+'/Patient_'+str(i)+'_'+str(l)+'_image.png', img_array)
        l=l+1

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
cd train_2d/

In [ ]:
path_lbl = 'labels1/'
path_img = 'ct_images/'

## Data

In [ ]:
fnames = get_image_files(path_img)
fnames[:3]

In [ ]:
lbl_names = get_image_files(path_lbl)
lbl_names[:3]

In [ ]:
img_f = fnames[4]
img = open_image(img_f)
img.show(figsize=(5,5))

In [ ]:
get_y_fn = lambda x:path_lbl+f'{(x.stem)}_label{x.suffix}'

In [ ]:
arr=np.load('Patient_01_100_image_label.npy')
x=plt.imshow(arr, cmap='Greys')

In [ ]:
mask = open_mask(get_y_fn(img_f))
mask.show(figsize=(5,5), alpha=1)

In [ ]:
src_size = np.array(mask.shape[1:])
src_size,mask.data

In [ ]:
codes = np.loadtxt('./codes.txt', dtype=str);codes#= np.delete(codes,4);codes

## Datasets

In [ ]:
size = src_size

free = gpu_mem_get_free_no_cache()
# the max size of bs depends on the available GPU RAM
if free > 8200: bs=8
else:           bs=4
print(f"using bs={bs}, have {free}MB of GPU RAM free")
bs=2

In [ ]:
free = gpu_mem_get_free_no_cache()
free

In [ ]:
src = (SegmentationItemList.from_folder(path_img)
       .split_by_rand_pct()
       .label_from_func(get_y_fn, classes=codes))

In [ ]:
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(imagenet_stats))

In [ ]:
data.show_batch(2, figsize=(10,7))

In [ ]:
data.show_batch(2, figsize=(10,7), ds_type=DatasetType.Valid)

## Model

In [ ]:

def dice_coeff(input,target):
    """
    input is a torch variable of size BatchxnclassesxHxW representing log probabilities for each class
    target is a 1-hot representation of the groundtruth, shoud have same size as the input
    """
    assert input.size() == target.size(), "Input sizes must be equal."
    assert input.dim() == 4, "Input must be a 4D Tensor."
    uniques=np.unique(target.numpy())
    assert set(list(uniques))<=set([0,1]), "target must only contain zeros and ones"

    probs=F.softmax(input)
    num=probs*target#b,c,h,w--p*g
    num=torch.sum(num,dim=3)#b,c,h
    num=torch.sum(num,dim=2)
    

    den1=torch.sum(den1,dim=3)#b,c,h
    den1=torch.sum(den1,dim=2)
    

    den2=target*target#--g^2
    den2=torch.sum(den2,dim=3)#b,c,h
    den2=torch.sum(den2,dim=2)#b,c
    

    dice=2*(num/(den1+den2))
    dice_eso=dice[:,1:]#we ignore bg dice val, and take the fg

    dice_total=-1*torch.sum(dice_eso)/dice_eso.size(0)#divide by batch_sz

    return dice_total
def iou(target,prediction):
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score
  
def soft_dice_loss(y_true, y_pred, epsilon=1e-6): 
    ''' 
    Soft dice loss calculation for arbitrary batch size, number of classes, and number of spatial dimensions.
    Assumes the `channels_last` format.
  
    # Arguments
        y_true: b x X x Y( x Z...) x c One hot encoding of ground truth
        y_pred: b x X x Y( x Z...) x c Network output, must sum to 1 over c channel (such as after softmax) 
        epsilon: Used for numerical stability to avoid divide by zero errors
    
    # References
        V-Net: Fully Convolutional Neural Networks for Volumetric Medical Image Segmentation 
        https://arxiv.org/abs/1606.04797
        More details on Dice loss formulation 
        https://mediatum.ub.tum.de/doc/1395260/1395260.pdf (page 72)
        
        Adapted from https://github.com/Lasagne/Recipes/issues/99#issuecomment-347775022
    '''
    
    # skip the batch and class axis for calculating Dice score
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * np.sum(y_pred * y_true, axes)
    denominator = np.sum(np.square(y_pred) + np.square(y_true), axes)
    
    return 1 - np.mean(numerator / (denominator + epsilon)) # average over classes and batch
name2id = {v:k for k,v in enumerate(codes)}
void_code = name2id['Void']

def acc(input, target):
    target = target.squeeze(1)
    mask = target != void_code
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()


In [ ]:
metrics=acc

In [ ]:
wd=1e-2

In [ ]:
learn = unet_learner(data, models.resnet18, metrics=metrics, wd=wd)

In [ ]:
from torchsummary import summary
summary(learn.model, input_size=(3, 512, 512))

In [ ]:
learn.model

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr=3e-3

In [ ]:
learn.fit_one_cycle(10, slice(lr), pct_start=0.9)

In [ ]:
learn.save('stage-1-imp')

In [ ]:
learn.load('stage-1-imp');

In [ ]:
learn.show_results(rows=3, figsize=(8,9))

## POST-PROCESSING

In [ ]:
img = open_image('./testing_2d/Patient_41/Patient_41_1_image.png')
img

In [ ]:
im_g=learn.predict(img)

In [ ]:
na1=(np.array(im_g[1][0]))
plt.imshow(arr)
plt.show()

In [ ]:
np.array(im_g[1][0])

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
for i in range (41,61):
    j=0
    l=1
    if(i<10):
        os.mkdir('./testing/Patient_0'+str(i))
        for k in os.listdir('./data/data_npy/Patient_0'+str(i)):
            j=j+1
    else:
        os.mkdir('./testing/Patient_'+str(i))
        for k in os.listdir('./testing_2d/Patient_'+str(i)):
            j=j+1
    while(l<=(j)):
        if(i<10):
            img = open_image('./testing_2d/Patient_0'+str(i)+'/Patient_0'+str(i)+'_'+str(l)+'_image.png')
            im_g=learn.predict(img)
            img = Image.fromarray(np.array(im_g[1][0]))
            matplotlib.image.imsave('./testing/Patient_0'+str(i)+'/Patient_0'+str(i)+'_'+str(l)+'_image_label.png', np.array(im_g[1][0]))
        else:
            img = open_image('./testing_2d/Patient_'+str(i)+'/Patient_'+str(i)+'_'+str(l)+'_image.png')
            im_g=learn.predict(img)
            img = Image.fromarray(np.array(im_g[1][0]))
            matplotlib.image.imsave('./testing/Patient_'+str(i)+'/Patient_'+str(i)+'_'+str(l)+'_image_label.png', np.array(im_g[1][0]))
        l=l+1
            

In [ ]:
import nibabel as nib

In [ ]:
import nibabel as nib
for i in range(42,61):
    j=0
    l=1
    img = open_image('./testing_2d/Patient_'+str(i)+'/Patient_'+str(i)+'_'+str(1)+'_image.png')
    im_g=learn.predict(img)
    na=np.array(im_g[1][0])
    for k in os.listdir('./testing_2d/Patient_'+str(i)):
        j=j+1
    while(l<(j)):
        img = open_image('./testing_2d/Patient_'+str(i)+'/Patient_'+str(i)+'_'+str(l+1)+'_image.png')
        im_g=learn.predict(img)
        a=np.array(im_g[1][0])
        na=np.dstack((na,a))
        l=l+1
    ex="test/Patient_"+str(i)+".nii"
    img = nib.load(ex)
    im1=np.array(img.get_affine())
    new_image = nib.Nifti1Image(np.asarray(na,dtype="uint8" ), affine=im1)
    nib.save(new_image,'./testing3d/Patient_'+str(i)+'_GT.nii.gz')

In [ ]:
na.shape

In [ ]:
learn.unfreeze()

In [ ]:
lrs = slice(lr/400,lr/4)

In [ ]:
learn.fit_one_cycle(12, lrs, pct_start=0.8)

In [ ]:
learn.save('stage-2');

## Go big

You may have to restart your kernel and come back to this stage if you run out of memory, and may also need to decrease `bs`.

In [ ]:
#learn.destroy() # uncomment once 1.0.46 is out

size = src_size

free = gpu_mem_get_free_no_cache()
# the max size of bs depends on the available GPU RAM
if free > 8200: bs=3
else:           bs=1
print(f"using bs={bs}, have {free}MB of GPU RAM free")

In [ ]:
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(imagenet_stats))

In [ ]:
learn = unet_learner(data, models.resnet50, metrics=metrics, wd=wd)

In [ ]:
learn.load('stage-2');

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr=1e-3

In [ ]:
learn.fit_one_cycle(10, slice(lr), pct_start=0.8)

In [ ]:
learn.save('stage-1-big')

In [ ]:
learn.load('stage-1-big');

In [ ]:
learn.unfreeze()

In [ ]:
lrs = slice(1e-6,lr/10)

In [ ]:
learn.fit_one_cycle(10, lrs)

In [ ]:
learn.save('stage-2-big')

In [ ]:
learn.load('stage-2-big');

In [ ]:
learn.show_results(rows=3, figsize=(10,10))

## VISUALIZING Axial, Sagittal and Coronal

In [ ]:
nii='Patient_41_GT.nii.gz'

In [ ]:
import nilearn
from nilearn import plotting
plotting.plot_stat_map(nii)

In [ ]:
nii='GT.nii.gz'

In [ ]:
import nilearn
from nilearn import plotting
plotting.plot_stat_map(nii)